In [204]:
import pandas as pd
import terality as te
# from codetiming import Timer
import random
from datetime import datetime


In [205]:
file_path = "C:\\\\Users\\\\bhask\\\\Google Drive\\\\datasets\\\\Chicago_taxi_trips\\\\"
timer = Timer(name="Terality Tests")

In [206]:
# Disabling Terality cache for testing
# cache_disabler = te.disable_cache()
# cache_disabler.__enter__()

In [207]:
def get_slice_range(limit):
    start = random.randint(1, limit)
    end = random.randint(start, start + 1000)
    return start, end

In [208]:
def get_time():
    return datetime.now()

In [209]:
def get_time_diff(start_time, end_time):
    return (end_time - start_time).total_seconds()

In [210]:
# A list of dataframe columns renamed by converting to lowercase and replacing spaces with '_'
columns = [
'trip_id',
'taxi_id',
'trip_start_timestamp',
'trip_end_timestamp',
'trip_seconds',
'trip_miles',
'pickup_census_tract',
'dropoff_census_tract',
'pickup_community_area',
'dropoff_community_area',
'fare',
'tips',
'tolls',
'extras',
'trip_total',
'payment_type',
'company',
'pickup_centroid_latitude',
'pickup_centroid_longitude',
'pickup_centroid_location',
'dropoff_centroid_latitude',
'dropoff_centroid_longitude',
'dropoff_centroid__location'
]

In [211]:
# Loading data into dataframes using Pandas
start_time = get_time()
taxi_trips_pdf = pd.read_csv(file_path + "Taxi_Trips_2019_2020.csv")
print(get_time_diff(start_time, get_time()))

355.504711


In [156]:
# Loading data into dataframes using Terality
start_time = get_time()
taxi_trips_tdf = te.read_csv(file_path + "Taxi_Trips_2019_2020.csv")
print(get_time_diff(start_time, get_time()))

C:\\Users\\bhask\\Google Drive\\datasets\\Chicago_taxi_trips\\Taxi_Trips_2019_2020.csv: 100%|█| 8.48G/8.48G [26:14<00:0


Elapsed time: 1664.1247 seconds


1664.1247297000082

In [157]:
start_time = get_time()
taxi_trips_pdf.head()
print(get_time_diff(start_time, get_time()))

Elapsed time: 0.0256 seconds


0.025595099999918602

In [158]:
start_time = get_time()
taxi_trips_tdf.head()
print(get_time_diff(start_time, get_time()))

Elapsed time: 4.2939 seconds


4.293879599994398

In [159]:
# Rename the columns of the Pandas dataframe
start_time = get_time()
taxi_trips_pdf.columns = columns
print(get_time_diff(start_time, get_time()))

Elapsed time: 0.0326 seconds


0.03256360000523273

In [160]:
# Rename the columns of the Terality dataframe
start_time = get_time()
taxi_trips_tdf.columns = columns
print(get_time_diff(start_time, get_time()))

Elapsed time: 2.7427 seconds


2.7426604999927804

In [161]:
# Row count before cleanup - using Pandas
start_time = get_time()
print("Pandas DF Row Count before: % d" % taxi_trips_pdf.shape[0])
print(get_time_diff(start_time, get_time()))

Pandas DF Row Count before:  20366397
Elapsed time: 0.0056 seconds


0.005633799999486655

In [162]:
# Remove all the rows with NaN values using Pandas. There is plenty of data even otherwise
start_time = get_time()
taxi_trips_pdf.dropna(how='any',axis=0,inplace=True)
print(get_time_diff(start_time, get_time()))

Elapsed time: 55.0317 seconds


55.03170560000581

In [163]:
# Row count after cleanup - using Pandas
start_time = get_time()
row_count = taxi_trips_pdf.shape[0]
print("Pandas DF Row Count after: % d" % row_count)
print(get_time_diff(start_time, get_time()))

Pandas DF Row Count after:  12199631
Elapsed time: 0.0024 seconds


0.0023604000016348436

In [164]:
# Row count before cleanup - using Terality
start_time = get_time()
print("Terality DF Row Count after: % d" % taxi_trips_tdf.shape[0])
print(get_time_diff(start_time, get_time()))

Terality DF Row Count after:  20366397
Elapsed time: 1.5089 seconds


1.5089397999981884

In [165]:
# Remove all the rows with NaN values using Terality. There is plenty of data even otherwise
start_time = get_time()
taxi_trips_tdf.dropna(how='any',axis=0,inplace=True)
print(get_time_diff(start_time, get_time()))

Elapsed time: 12.4042 seconds


12.404166700012865

In [166]:
# Row count after cleanup - using Terality
start_time = get_time()
print("Terality DF Row Count after: % d" % taxi_trips_tdf.shape[0])
print(get_time_diff(start_time, get_time()))

Terality DF Row Count after:  12199631
Elapsed time: 0.6453 seconds


0.6452872000081697

In [167]:
# Adding a new column to store datetime version of string datetimes - Pandas
start_time = get_time()
taxi_trips_pdf['trip_start_dt'] = pd.to_datetime(taxi_trips_pdf['trip_start_timestamp'], infer_datetime_format=True)
taxi_trips_pdf['trip_end_dt'] = pd.to_datetime(taxi_trips_pdf['trip_end_timestamp'], infer_datetime_format=True)
taxi_trips_pdf['trip_start_ym'] = taxi_trips_pdf['trip_start_dt'].dt.to_period('M')
taxi_trips_pdf['trip_end_ym'] = taxi_trips_pdf['trip_end_dt'].dt.to_period('M')
print(get_time_diff(start_time, get_time()))

Elapsed time: 15.0774 seconds


15.077363200005493

In [168]:
taxi_trips_pdf.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid__location,trip_start_dt,trip_end_dt,trip_start_ym,trip_end_ym
16,e066e483f0fc2a583895d24c9dec9864a6608052,4794c36af4c1852d90b5c8cf5f9f6cd38a8b9c1b68cf48...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,600.0,0.0,1.703108e+10,1.703184e+10,8.0,32.0,...,41.891972,-87.612945,POINT (-87.6129454143 41.8919715078),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),2019-01-01,2019-01-01 00:15:00,2019-01,2019-01
19,5431595d222a3e00a9e4b3e93c928c306dc71be3,86b0677bb9bcda0454863c70a616d1adc45129237848d1...,01/01/2019 12:00:00 AM,01/01/2019 12:00:00 AM,120.0,0.3,1.703184e+10,1.703132e+10,32.0,32.0,...,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.877406,-87.621972,POINT (-87.6219716519 41.8774061234),2019-01-01,2019-01-01 00:00:00,2019-01,2019-01
20,509e870adce2a514a4287e1a305fe4ddabc59836,38f6145c9a2b848dc1baa16fd91087e606b12bcb8757a9...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,360.0,0.8,1.703108e+10,1.703108e+10,8.0,8.0,...,41.898332,-87.620763,POINT (-87.6207628651 41.8983317935),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),2019-01-01,2019-01-01 00:15:00,2019-01,2019-01
22,31004535c12d4372c165e34386cde0521a0a72e8,ee8a2f86372e99ecf87d44e13a20ebda7db8ae69d5b444...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,360.0,1.0,1.703108e+10,1.703108e+10,8.0,8.0,...,41.890922,-87.618868,POINT (-87.6188683546 41.8909220259),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),2019-01-01,2019-01-01 00:15:00,2019-01,2019-01
23,da18ec450c1fed02fbbc675faa621f8ac50ab5dd,0013da5489fe976daf4f4a7d246073ecf2caed9b12b3be...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,420.0,0.8,1.703108e+10,1.703108e+10,8.0,8.0,...,41.898332,-87.620763,POINT (-87.6207628651 41.8983317935),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),2019-01-01,2019-01-01 00:15:00,2019-01,2019-01


In [169]:
# Adding a new column to store datetime version of string datetimes - Terality
start_time = get_time()
taxi_trips_tdf['trip_start_dt'] = te.to_datetime(taxi_trips_tdf['trip_start_timestamp'], infer_datetime_format=True)
taxi_trips_tdf['trip_end_dt'] = te.to_datetime(taxi_trips_tdf['trip_end_timestamp'], infer_datetime_format=True)
taxi_trips_tdf['trip_start_ym'] = taxi_trips_tdf['trip_start_dt'].dt.to_period('M')
taxi_trips_tdf['trip_end_ym'] = taxi_trips_tdf['trip_end_dt'].dt.to_period('M')
print(get_time_diff(start_time, get_time()))

Elapsed time: 21.8540 seconds


21.853961400003755

In [170]:
taxi_trips_tdf.head()
# print(get_time_diff(start_time, get_time()))


,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid__location,trip_start_dt,trip_end_dt,trip_start_ym,trip_end_ym
16,e066e483f0fc2a583895d24c9dec9864a6608052,4794c36af4c1852d90b5c8cf5f9f6cd38a8b9c1b68cf48...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,600.0,0.0,1.703108e+10,1.703184e+10,8.0,32.0,...,41.891972,-87.612945,POINT (-87.6129454143 41.8919715078),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),2019-01-01,2019-01-01 00:15:00,2019-01,2019-01
19,5431595d222a3e00a9e4b3e93c928c306dc71be3,86b0677bb9bcda0454863c70a616d1adc45129237848d1...,01/01/2019 12:00:00 AM,01/01/2019 12:00:00 AM,120.0,0.3,1.703184e+10,1.703132e+10,32.0,32.0,...,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.877406,-87.621972,POINT (-87.6219716519 41.8774061234),2019-01-01,2019-01-01 00:00:00,2019-01,2019-01
20,509e870adce2a514a4287e1a305fe4ddabc59836,38f6145c9a2b848dc1baa16fd91087e606b12bcb8757a9...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,360.0,0.8,1.703108e+10,1.703108e+10,8.0,8.0,...,41.898332,-87.620763,POINT (-87.6207628651 41.8983317935),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),2019-01-01,2019-01-01 00:15:00,2019-01,2019-01
22,31004535c12d4372c165e34386cde0521a0a72e8,ee8a2f86372e99ecf87d44e13a20ebda7db8ae69d5b444...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,360.0,1.0,1.703108e+10,1.703108e+10,8.0,8.0,...,41.890922,-87.618868,POINT (-87.6188683546 41.8909220259),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),2019-01-01,2019-01-01 00:15:00,2019-01,2019-01
23,da18ec450c1fed02fbbc675faa621f8ac50ab5dd,0013da5489fe976daf4f4a7d246073ecf2caed9b12b3be...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,420.0,0.8,1.703108e+10,1.703108e+10,8.0,8.0,...,41.898332,-87.620763,POINT (-87.6207628651 41.8983317935),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),2019-01-01,2019-01-01 00:15:00,2019-01,2019-01


In [171]:
# Describe - Pandas
start_time = get_time()
taxi_trips_pdf.describe()

,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude
count,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07
mean,7.806322e+02,2.882408e+00,1.703141e+10,1.703138e+10,2.628359e+01,2.319189e+01,1.279652e+01,1.781760e+00,7.331033e-04,8.851525e-01,1.560699e+01,4.189265e+01,-8.765359e+01,4.189144e+01,-8.764459e+01
std,1.207102e+03,5.479885e+00,3.436317e+05,3.341613e+05,1.881160e+01,1.634532e+01,6.189516e+01,2.826719e+00,3.181317e-01,1.609252e+01,6.459551e+01,3.113462e-02,7.520429e-02,2.604625e-02,5.721153e-02
min,0.000000e+00,0.000000e+00,1.703101e+10,1.703101e+10,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.165022e+01,-8.790304e+01,4.166376e+01,-8.790304e+01
25%,3.400000e+02,5.400000e-01,1.703108e+10,1.703108e+10,8.000000e+00,8.000000e+00,5.750000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+00,4.188099e+01,-8.763784e+01,4.188099e+01,-8.764090e+01
50%,5.140000e+02,1.080000e+00,1.703132e+10,1.703128e+10,2.800000e+01,2.800000e+01,7.250000e+00,9.000000e-01,0.000000e+00,0.000000e+00,9.000000e+00,4.188530e+01,-8.763186e+01,4.188530e+01,-8.763186e+01
75%,8.230000e+02,1.900000e+00,1.703184e+10,1.703184e+10,3.200000e+01,3.200000e+01,1.025000e+01,2.000000e+00,0.000000e+00,1.000000e+00,1.250000e+01,4.189322e+01,-8.762197e+01,4.189503e+01,-8.762197e+01
max,8.640000e+04,9.959000e+02,1.703198e+10,1.703198e+10,7.700000e+01,7.700000e+01,9.800260e+03,4.000000e+02,9.606800e+02,9.327760e+03,9.800300e+03,4.202122e+01,-8.753071e+01,4.202122e+01,-8.753071e+01


In [172]:
print(get_time_diff(start_time, get_time()))

Elapsed time: 6.6500 seconds


6.650022000001627

In [173]:
# Describe - Terality
start_time = get_time()
taxi_trips_tdf.describe()

,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude
count,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07
mean,7.806322e+02,2.882408e+00,1.703141e+10,1.703138e+10,2.628359e+01,2.319189e+01,1.279652e+01,1.781760e+00,7.331033e-04,8.851525e-01,1.560699e+01,4.189265e+01,-8.765359e+01,4.189144e+01,-8.764459e+01
std,1.207102e+03,5.479885e+00,3.436317e+05,3.341613e+05,1.881160e+01,1.634532e+01,6.189516e+01,2.826719e+00,3.181317e-01,1.609252e+01,6.459551e+01,3.113462e-02,7.520429e-02,2.604625e-02,5.721153e-02
min,0.000000e+00,0.000000e+00,1.703101e+10,1.703101e+10,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.165022e+01,-8.790304e+01,4.166376e+01,-8.790304e+01
25%,3.400000e+02,5.400000e-01,1.703108e+10,1.703108e+10,8.000000e+00,8.000000e+00,5.750000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+00,4.188099e+01,-8.763784e+01,4.188099e+01,-8.764090e+01
50%,5.140000e+02,1.080000e+00,1.703132e+10,1.703128e+10,2.800000e+01,2.800000e+01,7.250000e+00,9.000000e-01,0.000000e+00,0.000000e+00,9.000000e+00,4.188530e+01,-8.763186e+01,4.188530e+01,-8.763186e+01
75%,8.230000e+02,1.900000e+00,1.703184e+10,1.703184e+10,3.200000e+01,3.200000e+01,1.025000e+01,2.000000e+00,0.000000e+00,1.000000e+00,1.250000e+01,4.189322e+01,-8.762197e+01,4.189503e+01,-8.762197e+01
max,8.640000e+04,9.959000e+02,1.703198e+10,1.703198e+10,7.700000e+01,7.700000e+01,9.800260e+03,4.000000e+02,9.606800e+02,9.327760e+03,9.800300e+03,4.202122e+01,-8.753071e+01,4.202122e+01,-8.753071e+01


In [174]:
print(get_time_diff(start_time, get_time()))

Elapsed time: 21.4492 seconds


21.449159700001474

In [175]:
# Query 1: Number of trips grouped by taxi - Using Pandas
start_time = get_time()
taxi_trips_pdf.groupby('taxi_id').count()['trip_id']


taxi_id
0008de7a146802839c9e6059f482d292ebdae13c5c31dd6e5983a80882e2a5dbcd6ea098c2fcd56f34ce02645eb94c6b39512e9304837746d4e289b6236c2c5b    3372
001330b81e23412049f9c3eff5b6e972a91afe59c9aa36ef29747881b4bf803adfd02b1a71cca5cb8c2214275330aa4520df34d7848100414acb4b4cad642e62    3072
0013da5489fe976daf4f4a7d246073ecf2caed9b12b3be70eff37b63ef9ca4102c972145ddd8537811752d51b222b4618dfbae451b966d7a3ad9afa4ec878a6e    4093
0041f8f0c91881c1e1913f2548522495fe3c4c719aa67fd2483ebbcc22e0aa1bfff2cddb32c35cde3ad9766b0e8d44c04ff5dfdcf1035462ba81a13e3ba6b8f9    3431
0044e6c0d091476299b99345501f756b23632a96cbaf40e872fbf14f976410d3f938aacc643ed608b2aa42809222d4458e1aab5e0848e9b952f35616785c3a36    2104
                                                                                                                                    ... 
ffd70d111c7da2e5eff27ad900c8eb94d74fe286bb7aa58029ee2baac3f3b310dea5da2a6a77a366b7b6d4b104b91ef75e7959ac6b1f558be7844417cbe3e12f     603
ffda53354c610fd3af1aee46d723028a4

In [176]:
print(get_time_diff(start_time, get_time()))

Elapsed time: 10.4241 seconds


10.42411429999629

In [177]:
# Query 1: Number of trips grouped by taxi - Using Terality
start_time = get_time()
taxi_trips_pdf.groupby('taxi_id').count()['trip_id']



taxi_id
0008de7a146802839c9e6059f482d292ebdae13c5c31dd6e5983a80882e2a5dbcd6ea098c2fcd56f34ce02645eb94c6b39512e9304837746d4e289b6236c2c5b    3372
001330b81e23412049f9c3eff5b6e972a91afe59c9aa36ef29747881b4bf803adfd02b1a71cca5cb8c2214275330aa4520df34d7848100414acb4b4cad642e62    3072
0013da5489fe976daf4f4a7d246073ecf2caed9b12b3be70eff37b63ef9ca4102c972145ddd8537811752d51b222b4618dfbae451b966d7a3ad9afa4ec878a6e    4093
0041f8f0c91881c1e1913f2548522495fe3c4c719aa67fd2483ebbcc22e0aa1bfff2cddb32c35cde3ad9766b0e8d44c04ff5dfdcf1035462ba81a13e3ba6b8f9    3431
0044e6c0d091476299b99345501f756b23632a96cbaf40e872fbf14f976410d3f938aacc643ed608b2aa42809222d4458e1aab5e0848e9b952f35616785c3a36    2104
                                                                                                                                    ... 
ffd70d111c7da2e5eff27ad900c8eb94d74fe286bb7aa58029ee2baac3f3b310dea5da2a6a77a366b7b6d4b104b91ef75e7959ac6b1f558be7844417cbe3e12f     603
ffda53354c610fd3af1aee46d723028a4

In [178]:
print(get_time_diff(start_time, get_time()))

Elapsed time: 9.7827 seconds


9.782682299992302

In [179]:
# Query 2: Number of trips grouped by taxi and year/month - Using Pandas
start_time = get_time()
taxi_trips_pdf.groupby(['taxi_id', 'trip_start_ym']).count()['trip_id']

taxi_id                                                                                                                           trip_start_ym
0008de7a146802839c9e6059f482d292ebdae13c5c31dd6e5983a80882e2a5dbcd6ea098c2fcd56f34ce02645eb94c6b39512e9304837746d4e289b6236c2c5b  2019-01          230
                                                                                                                                  2019-02          248
                                                                                                                                  2019-03          212
                                                                                                                                  2019-04          202
                                                                                                                                  2019-05          270
                                                                                                     

In [180]:
print(get_time_diff(start_time, get_time()))

Elapsed time: 12.7381 seconds


12.738141200010432

In [181]:
# Query 2: Number of trips grouped by taxi and year/month - Using Terality
start_time = get_time()
taxi_trips_tdf.groupby(['taxi_id', 'trip_start_ym']).count()['trip_id']

taxi_id                                                                                                                           trip_start_ym
0008de7a146802839c9e6059f482d292ebdae13c5c31dd6e5983a80882e2a5dbcd6ea098c2fcd56f34ce02645eb94c6b39512e9304837746d4e289b6236c2c5b  2019-01          230
                                                                                                                                  2019-02          248
                                                                                                                                  2019-03          212
                                                                                                                                  2019-04          202
                                                                                                                                  2019-05          270
                                                                                                     

In [182]:
print(get_time_diff(start_time, get_time()))

Elapsed time: 21.8460 seconds


21.84595960000297

In [183]:
# Query 3: Number of trips by Taxi Company - using Pandas
start_time = get_time()
taxi_trips_pdf.groupby('company').count()['trip_id'].sort_values(ascending=False).head(10)


company
Taxi Affiliation Services            2769009
Chicago Carriage Cab Corp            1506931
Flash Cab                            1220891
Sun Taxi                             1046724
Medallion Leasin                      932280
City Service                          906870
Star North Management LLC             742670
Blue Ribbon Taxi Association Inc.     581908
Choice Taxi Association               400096
Taxicab Insurance Agency, LLC         340868
Name: trip_id, dtype: int64

In [184]:
print(get_time_diff(start_time, get_time()))

Elapsed time: 6.2821 seconds


6.282142600000952

In [185]:
# Query 3: Number of trips by Taxi Company - using Terality
start_time = get_time()
taxi_trips_tdf.groupby('company').count()['trip_id'].sort_values(ascending=False).head(10)


company
Taxi Affiliation Services            2769009
Chicago Carriage Cab Corp            1506931
Flash Cab                            1220891
Sun Taxi                             1046724
Medallion Leasin                      932280
City Service                          906870
Star North Management LLC             742670
Blue Ribbon Taxi Association Inc.     581908
Choice Taxi Association               400096
Taxicab Insurance Agency, LLC         340868
Name: trip_id, dtype: int64

In [186]:
print(get_time_diff(start_time, get_time()))

Elapsed time: 26.4996 seconds


26.49955360000604

In [187]:
# Query 4: Sort by Taxi Company - using Pandas
start_time = get_time()
taxi_trips_pdf.sort_values(by="company")
print(get_time_diff(start_time, get_time()))

Elapsed time: 14.3080 seconds


14.307979899997008

In [188]:
# Query 4: Sort by Taxi Company - using Terality
start_time = get_time()
taxi_trips_tdf.sort_values(by="company")
print(get_time_diff(start_time, get_time()))

Elapsed time: 17.9864 seconds


17.986363899995922

In [189]:
range = get_slice_range(row_count)
print("Start: %d, End: %d" % range)


Start: 11726145, End: 11726587


In [190]:
# Query 5: Slicing using random numbers for start and end - using Pandas
start_time = get_time()
taxi_trips_pdf[range[0]:range[1]]

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid__location,trip_start_dt,trip_end_dt,trip_start_ym,trip_end_ym
18534701,9a088fe5e973ea5bfaa2436e4291dfbf464fd5d8,ed5ea8aa629c794c3ecfef8ef582d7b6077a7a4c25e000...,02/26/2020 06:15:00 PM,02/26/2020 06:15:00 PM,359.0,0.78,1.703132e+10,1.703184e+10,32.0,32.0,...,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),2020-02-26 18:15:00,2020-02-26 18:15:00,2020-02,2020-02
18534703,99733028b7fed5c52fb86b0274e64dcb12307c7c,f8e444c5987ebb8995986108b5cbb0ebade565cf6a9e3f...,02/26/2020 06:15:00 PM,02/26/2020 06:15:00 PM,360.0,0.50,1.703184e+10,1.703128e+10,32.0,28.0,...,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.879255,-87.642649,POINT (-87.642648998 41.8792550844),2020-02-26 18:15:00,2020-02-26 18:15:00,2020-02,2020-02
18534704,99947395f306b17eeb50a9ffbbe11de26f5a4cf1,ba48aac1be874e3212623beb83b4538c5567a8676f2cf3...,02/26/2020 06:15:00 PM,02/26/2020 06:30:00 PM,1260.0,2.70,1.703184e+10,1.703107e+10,32.0,7.0,...,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.914616,-87.631717,POINT (-87.6317173661 41.9146162864),2020-02-26 18:15:00,2020-02-26 18:30:00,2020-02,2020-02
18534705,98948064165785d24b8d87bc0d01bfa9facc1f8f,99cde37aeabdaa069a6b277bae9976d94d0095aba59302...,02/26/2020 06:15:00 PM,02/26/2020 06:15:00 PM,480.0,1.00,1.703184e+10,1.703184e+10,32.0,32.0,...,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),2020-02-26 18:15:00,2020-02-26 18:15:00,2020-02,2020-02
18534708,985b84075003d4b7831418e43553039c079f0eae,b884dd3799982951b7b1293d92a3e6e90ba3483a6fdbf2...,02/26/2020 06:15:00 PM,02/26/2020 06:30:00 PM,623.0,1.59,1.703184e+10,1.703108e+10,32.0,8.0,...,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.899156,-87.626211,POINT (-87.6262105324 41.8991556134),2020-02-26 18:15:00,2020-02-26 18:30:00,2020-02,2020-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18535323,41dd5d8f91b4de1deb5adf1d521bbf27327c273e,050533f7d5b99dfd788daf40b484fb43fbb69aeb8c6231...,02/26/2020 06:30:00 PM,02/26/2020 07:00:00 PM,1073.0,3.49,1.703128e+10,1.703107e+10,28.0,7.0,...,41.885300,-87.642808,POINT (-87.6428084655 41.8853000224),41.922083,-87.634156,POINT (-87.6341560931 41.922082541),2020-02-26 18:30:00,2020-02-26 19:00:00,2020-02,2020-02
18535324,42ebba5e8a082b6c8800904b0a6cdedb7a9fffeb,e8b30fe3cdcf458994b6943ba607e06f31b92202cab6b7...,02/26/2020 06:30:00 PM,02/26/2020 06:30:00 PM,273.0,0.81,1.703108e+10,1.703128e+10,8.0,28.0,...,41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),41.879255,-87.642649,POINT (-87.642648998 41.8792550844),2020-02-26 18:30:00,2020-02-26 18:30:00,2020-02,2020-02
18535325,406e80a702262eb0bdef0b4515f702977dcc19df,034ec4437f304cffcd4acf850106574343e69d3c5424ac...,02/26/2020 06:30:00 PM,02/26/2020 06:30:00 PM,525.0,1.29,1.703128e+10,1.703108e+10,28.0,8.0,...,41.879255,-87.642649,POINT (-87.642648998 41.8792550844),41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),2020-02-26 18:30:00,2020-02-26 18:30:00,2020-02,2020-02
18535327,4268219cee2844941d3c284a1c4909b58af81011,59cfe5aef1ecdd4418c373fe7848fb1f9defcbcfe70f17...,02/26/2020 06:30:00 PM,02/26/2020 07:15:00 PM,3325.0,2.64,1.703108e+10,1.703107e+10,8.0,7.0,...,41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),41.929263,-87.635891,POINT (-87.6358909539 41.9292629902),2020-02-26 18:30:00,2020-02-26 19:15:00,2020-02,2020-02


In [191]:
print(get_time_diff(start_time, get_time()))

Elapsed time: 0.0570 seconds


0.05695739999646321

In [192]:
# Query 5: Slicing using random numbers for start and end - using Terality
start_time = get_time()
taxi_trips_tdf[range[0]:range[1]]

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid__location,trip_start_dt,trip_end_dt,trip_start_ym,trip_end_ym
18534701,9a088fe5e973ea5bfaa2436e4291dfbf464fd5d8,ed5ea8aa629c794c3ecfef8ef582d7b6077a7a4c25e000...,02/26/2020 06:15:00 PM,02/26/2020 06:15:00 PM,359.0,0.78,1.703132e+10,1.703184e+10,32.0,32.0,...,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),2020-02-26 18:15:00,2020-02-26 18:15:00,2020-02,2020-02
18534703,99733028b7fed5c52fb86b0274e64dcb12307c7c,f8e444c5987ebb8995986108b5cbb0ebade565cf6a9e3f...,02/26/2020 06:15:00 PM,02/26/2020 06:15:00 PM,360.0,0.50,1.703184e+10,1.703128e+10,32.0,28.0,...,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.879255,-87.642649,POINT (-87.642648998 41.8792550844),2020-02-26 18:15:00,2020-02-26 18:15:00,2020-02,2020-02
18534704,99947395f306b17eeb50a9ffbbe11de26f5a4cf1,ba48aac1be874e3212623beb83b4538c5567a8676f2cf3...,02/26/2020 06:15:00 PM,02/26/2020 06:30:00 PM,1260.0,2.70,1.703184e+10,1.703107e+10,32.0,7.0,...,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.914616,-87.631717,POINT (-87.6317173661 41.9146162864),2020-02-26 18:15:00,2020-02-26 18:30:00,2020-02,2020-02
18534705,98948064165785d24b8d87bc0d01bfa9facc1f8f,99cde37aeabdaa069a6b277bae9976d94d0095aba59302...,02/26/2020 06:15:00 PM,02/26/2020 06:15:00 PM,480.0,1.00,1.703184e+10,1.703184e+10,32.0,32.0,...,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),2020-02-26 18:15:00,2020-02-26 18:15:00,2020-02,2020-02
18534708,985b84075003d4b7831418e43553039c079f0eae,b884dd3799982951b7b1293d92a3e6e90ba3483a6fdbf2...,02/26/2020 06:15:00 PM,02/26/2020 06:30:00 PM,623.0,1.59,1.703184e+10,1.703108e+10,32.0,8.0,...,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.899156,-87.626211,POINT (-87.6262105324 41.8991556134),2020-02-26 18:15:00,2020-02-26 18:30:00,2020-02,2020-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18535323,41dd5d8f91b4de1deb5adf1d521bbf27327c273e,050533f7d5b99dfd788daf40b484fb43fbb69aeb8c6231...,02/26/2020 06:30:00 PM,02/26/2020 07:00:00 PM,1073.0,3.49,1.703128e+10,1.703107e+10,28.0,7.0,...,41.885300,-87.642808,POINT (-87.6428084655 41.8853000224),41.922083,-87.634156,POINT (-87.6341560931 41.922082541),2020-02-26 18:30:00,2020-02-26 19:00:00,2020-02,2020-02
18535324,42ebba5e8a082b6c8800904b0a6cdedb7a9fffeb,e8b30fe3cdcf458994b6943ba607e06f31b92202cab6b7...,02/26/2020 06:30:00 PM,02/26/2020 06:30:00 PM,273.0,0.81,1.703108e+10,1.703128e+10,8.0,28.0,...,41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),41.879255,-87.642649,POINT (-87.642648998 41.8792550844),2020-02-26 18:30:00,2020-02-26 18:30:00,2020-02,2020-02
18535325,406e80a702262eb0bdef0b4515f702977dcc19df,034ec4437f304cffcd4acf850106574343e69d3c5424ac...,02/26/2020 06:30:00 PM,02/26/2020 06:30:00 PM,525.0,1.29,1.703128e+10,1.703108e+10,28.0,8.0,...,41.879255,-87.642649,POINT (-87.642648998 41.8792550844),41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),2020-02-26 18:30:00,2020-02-26 18:30:00,2020-02,2020-02
18535327,4268219cee2844941d3c284a1c4909b58af81011,59cfe5aef1ecdd4418c373fe7848fb1f9defcbcfe70f17...,02/26/2020 06:30:00 PM,02/26/2020 07:15:00 PM,3325.0,2.64,1.703108e+10,1.703107e+10,8.0,7.0,...,41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),41.929263,-87.635891,POINT (-87.6358909539 41.9292629902),2020-02-26 18:30:00,2020-02-26 19:15:00,2020-02,2020-02


In [193]:
print(get_time_diff(start_time, get_time()))

Elapsed time: 3.9668 seconds


3.9667974000039976

In [194]:
# Query 6: Selecting a single row from an unindexed dataframe - using Pandas
start_time = get_time()
taxi_trips_pdf[taxi_trips_pdf['trip_id'] == 'd3e437e3af9c691e6a9fe1f9802605d989605fdb']

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid__location,trip_start_dt,trip_end_dt,trip_start_ym,trip_end_ym
13905408,d3e437e3af9c691e6a9fe1f9802605d989605fdb,ac66734513cb7344a5501fa22f825c940448a4041e0302...,10/30/2019 05:30:00 PM,10/30/2019 06:00:00 PM,1396.0,1.14,1.703108e+10,1.703108e+10,8.0,8.0,...,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),2019-10-30 17:30:00,2019-10-30 18:00:00,2019-10,2019-10


In [195]:
print(get_time_diff(start_time, get_time()))

Elapsed time: 0.8416 seconds


0.841567399998894

In [196]:
# Query 6: Selecting a single row from an unindexed dataframe - using Terality
start_time = get_time()
taxi_trips_tdf[taxi_trips_tdf['trip_id'] == 'd3e437e3af9c691e6a9fe1f9802605d989605fdb']

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid__location,trip_start_dt,trip_end_dt,trip_start_ym,trip_end_ym
13905408,d3e437e3af9c691e6a9fe1f9802605d989605fdb,ac66734513cb7344a5501fa22f825c940448a4041e0302...,10/30/2019 05:30:00 PM,10/30/2019 06:00:00 PM,1396.0,1.14,1.703108e+10,1.703108e+10,8.0,8.0,...,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),2019-10-30 17:30:00,2019-10-30 18:00:00,2019-10,2019-10


In [197]:
print(get_time_diff(start_time, get_time()))

Elapsed time: 9.3218 seconds


9.32180080001126

In [198]:
# Query 7: Range queries using isin() - Using Pandas
start_time = get_time()
taxi_trips_pdf[taxi_trips_pdf['company'].isin(['Taxi Affiliation Services', 'Blue Diamond', 'Chicago Taxicab', 'Nova Taxi Affiliation Llc', 'U Taxicab'])]

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid__location,trip_start_dt,trip_end_dt,trip_start_ym,trip_end_ym
20,509e870adce2a514a4287e1a305fe4ddabc59836,38f6145c9a2b848dc1baa16fd91087e606b12bcb8757a9...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,360.0,0.8,1.703108e+10,1.703108e+10,8.0,8.0,...,41.898332,-87.620763,POINT (-87.6207628651 41.8983317935),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),2019-01-01 00:00:00,2019-01-01 00:15:00,2019-01,2019-01
22,31004535c12d4372c165e34386cde0521a0a72e8,ee8a2f86372e99ecf87d44e13a20ebda7db8ae69d5b444...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,360.0,1.0,1.703108e+10,1.703108e+10,8.0,8.0,...,41.890922,-87.618868,POINT (-87.6188683546 41.8909220259),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),2019-01-01 00:00:00,2019-01-01 00:15:00,2019-01,2019-01
30,601ea33fd396dadf3c05667ce8badbab55968930,72cd7073f26565b4a99646e795893b87b06e8086ad9a49...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,180.0,0.0,1.703108e+10,1.703108e+10,8.0,8.0,...,41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),41.890922,-87.618868,POINT (-87.6188683546 41.8909220259),2019-01-01 00:00:00,2019-01-01 00:15:00,2019-01,2019-01
35,d1c76d15cec1a2ee58b408aff06e303aa86dd37f,d38890256d8a1e8146f0b15dc23e3b2a140b4fa3834698...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,300.0,0.0,1.703108e+10,1.703108e+10,8.0,8.0,...,41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),41.898332,-87.620763,POINT (-87.6207628651 41.8983317935),2019-01-01 00:00:00,2019-01-01 00:15:00,2019-01,2019-01
36,e933481970a4efe76e013706d4056104dbf3a599,3506f62ee1c70c02fe00b742679684b319d5b712182f82...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,360.0,0.0,1.703108e+10,1.703108e+10,8.0,8.0,...,41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),41.892073,-87.628874,POINT (-87.6288741572 41.8920726347),2019-01-01 00:00:00,2019-01-01 00:15:00,2019-01,2019-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20365970,8b2fb9ed7cb74a7645c95538373111ac90be222a,7476a493f34aa1075570e89104cdaf3aaf5c0fd4679382...,12/31/2020 08:45:00 PM,12/31/2020 09:00:00 PM,1200.0,17.1,1.703198e+10,1.703128e+10,76.0,28.0,...,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),41.879255,-87.642649,POINT (-87.642648998 41.8792550844),2020-12-31 20:45:00,2020-12-31 21:00:00,2020-12,2020-12
20366091,91fddaa11017287697ed65e5b6fb6399ea4f7ac4,5f88daef22c65f61d051338809a96d6bc7f471c4e2cd73...,12/31/2020 09:30:00 PM,12/31/2020 09:30:00 PM,120.0,0.2,1.703132e+10,1.703132e+10,32.0,32.0,...,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),2020-12-31 21:30:00,2020-12-31 21:30:00,2020-12,2020-12
20366290,debd80beca876e44a7d485a2fd2fd66816f8fc2b,42574f4a000f78c9ccbf2cfc282f31617096a9fea4a0d5...,12/31/2020 10:45:00 PM,12/31/2020 11:00:00 PM,180.0,0.0,1.703198e+10,1.703198e+10,76.0,76.0,...,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),2020-12-31 22:45:00,2020-12-31 23:00:00,2020-12,2020-12
20366292,e9fc3c32d17dd7dcd54cabaca1781d4684b8832d,be2af1b95b937dfe54bf9a4f4331745a673529a7edfab2...,12/31/2020 10:45:00 PM,12/31/2020 11:00:00 PM,480.0,2.8,1.703198e+10,1.703198e+10,76.0,76.0,...,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),2020-12-31 22:45:00,2020-12-31 23:00:00,2020-12,2020-12


In [199]:
print(get_time_diff(start_time, get_time()))

Elapsed time: 2.5322 seconds


2.532223599991994

In [200]:
# Query 7: Range queries using isin() - Using Terality
start_time = get_time()
taxi_trips_pdf['company'].isin(['Taxi Affiliation Services', 'Blue Diamond', 'Chicago Taxicab', 'Nova Taxi Affiliation Llc', 'U Taxicab'])

16          False
19          False
20           True
22           True
23          False
            ...  
20366129    False
20366142    False
20366290     True
20366292     True
20366309     True
Name: company, Length: 12199631, dtype: bool

In [201]:
print(get_time_diff(start_time, get_time()))

Elapsed time: 0.5331 seconds


0.5330955999961589